In [ ]:
# FP-Growth Algorithm  in Python

from collections import defaultdict, namedtuple

# Test Dataset
transactions = [
    ['Milk', 'Bread', 'Butter'],
    ['Bread', 'Butter'],
    ['Milk', 'Bread'],
    ['Milk', 'Butter'],
    ['Bread', 'Butter']
]

min_support = 2   # minimum support count


# FP Tree Node Definition
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None



# Building the FP-Tree

def build_fp_tree(transactions, min_support):
    # First scan: count item frequencies
    item_counts = defaultdict(int)
    for trans in transactions:
        for item in trans:
            item_counts[item] += 1

    # Remove items below minimum support
    item_counts = {item: count for item, count in item_counts.items()
                   if count >= min_support}

    # If no items meet support, stop
    if len(item_counts) == 0:
        return None, None

    # Create header table
    header_table = {item: [count, None] for item, count in item_counts.items()}

    # Create root of the FP-tree
    root = FPNode(None, 1, None)

    # Second scan: Build the tree
    for trans in transactions:
        # Filter and sort transactions by frequency
        filtered = [item for item in trans if item in item_counts]
        filtered.sort(key=lambda x: item_counts[x], reverse=True)

        # Insert into the FP tree
        current_node = root
        for item in filtered:
            if item not in current_node.children:
                new_node = FPNode(item, 1, current_node)
                current_node.children[item] = new_node

                # Update header table link
                if header_table[item][1] is None:
                    header_table[item][1] = new_node
                else:
                    node = header_table[item][1]
                    while node.link:
                        node = node.link
                    node.link = new_node
            else:
                current_node.children[item].count += 1

            current_node = current_node.children[item]

    return root, header_table


# Mine Frequent Patterns
def mine_fp_tree(header_table, min_support, prefix, frequent_patterns):
    # Process items in ascending frequency
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])

    for item, table_info in sorted_items:
        new_prefix = prefix + [item]
        frequent_patterns.append(new_prefix)

        # Build conditional pattern base
        conditional_patterns = []
        node = table_info[1]

        while node:
            path = []
            parent = node.parent

            # Trace to root
            while parent and parent.item is not None:
                path.append(parent.item)
                parent = parent.parent

            path.reverse()

            for _ in range(node.count):
                if len(path) > 0:
                    conditional_patterns.append(path)

            node = node.link

        # Build conditional tree
        conditional_tree, new_header = build_fp_tree(conditional_patterns, min_support)

        if new_header:
            mine_fp_tree(new_header, min_support, new_prefix, frequent_patterns)


# Run FP-Growth
root, header_table = build_fp_tree(transactions, min_support)
frequent_patterns = []

if header_table:
    mine_fp_tree(header_table, min_support, [], frequent_patterns)

# Display results
print("=== Frequent Patterns Found ===")
for pattern in frequent_patterns:
    print(pattern)


=== Frequent Patterns Found ===
['Milk']
['Milk', 'Bread']
['Milk', 'Butter']
['Bread']
['Butter']
['Butter', 'Bread']
